[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://www.kaggle.com/datasets/medali1992/beijing-air-quality-preprocessed)

In [ ]:
import os
import gc

import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

DATA_PATH = '/home/mohamed-ali/Downloads/Beijing air quality/data'
OUT_PATH = '/home/mohamed-ali/Downloads/Beijing air quality/data/'

In [ ]:
data_list = []

for path in os.listdir(DATA_PATH):
    data = pd.read_csv(os.path.join(DATA_PATH, path))
    data_list.append(data)
    
data = pd.concat(data_list, axis=0)

In [ ]:
pd.__version__

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
data.head()

In [ ]:
data.isna().sum()

In [ ]:
pm2_5 = data['PM2.5']
sns.histplot(pm2_5, kde=True)

In [ ]:
# Remove outliers for the 'LIMT_BAL' variable
q1 = data['PM2.5'].quantile(0.25) # First quartile
q3 = data['PM2.5'].quantile(0.75) # Third quartile
iqr = q3 - q1 # Interquartile range
# Filter outlier observation and take their index
filtration = (data['PM2.5'] < q1 - 1.5 * iqr) | (data['PM2.5'] > q3 + 1.5 *iqr)
df_index1 = list(data.loc[filtration].index)
data = data.loc[~filtration]

In [ ]:
data.shape

In [ ]:
CAT_FEATURES = data.select_dtypes(include=['object']).columns.tolist()

for cat_col in CAT_FEATURES:
    encoder = LabelEncoder()
    data[cat_col] = encoder.fit_transform(data[cat_col])

data['wd'].fillna(data['wd'].mode(), inplace=True)
assert data['wd'].isna().sum() == 0
    
del encoder, CAT_FEATURES
_ = gc.collect()

In [ ]:
%%time
knn_imputer = KNNImputer(n_neighbors=5)
na_columns = data.loc[:, data.isnull().any()].columns.tolist()
num_cols = [col for col in na_columns if col != 'wd']
data[num_cols] = knn_imputer.fit_transform(data[na_columns].values)

In [ ]:
data.isna().sum()

In [ ]:
data.to_csv(f'{OUT_PATH}airquality_data.csv', index=False)